In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
openpose_path = '/Users/andrei-macpro/Documents/Data/1042_meal/'

In [ ]:
poses = [x for x in sorted_alphanumeric(os.listdir(openpose_path))]
#poses = [int(x.split('.')[0].split("_")[1]) for x in poses]

In [ ]:
human_no = []
for frame in sorted_alphanumeric(os.listdir(openpose_path)):
    data = pd.read_csv(openpose_path + frame)
       # print(len(data.columns)-1)
    human_no.append((len(data.columns)-1)/3)
human_no

In [ ]:
## frames
def return_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while(cap.isOpened()):
            ret_val, image = cap.read()
            if ret_val == False:
                break
            frames.append(image)
            count = count+1
    return (frames)
frames = return_frames('/Users/andrei-macpro/Documents/Data/videoss/1042_meal.mp4')

In [ ]:
frames

In [ ]:
storage = [] ## store 10 frames
counter = 0
skeletons = []
for frame in sorted_alphanumeric(os.listdir(openpose_path)):
    data = pd.read_csv(openpose_path + frame)
    if (len(data.columns)-1)/3 == 1:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1'])]})
    elif (len(data.columns)-1)/3 == 2:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2'])]})


In [ ]:
len(skeletons)

In [ ]:
type(skeletons[0]['skeleton1'][0][1]), skeletons[0]['skeleton1'][1][1]

In [ ]:
skeletons[0]['skeleton1']

In [ ]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
new_skeleton2 = []
hooman1 = []
counter=0
hooman2_frames = []
for i in range(len(skeletons)): 

    if i == 0 and len(skeletons[i])==1:   # we start in the case where only one skeleton is available
        new_skeleton1.append(skeletons[i]['skeleton1'])
        continue
    if i==0 and len(skeletons[i])==2: ## if both skeletons are available 
        new_skeleton1.append(skeletons[i]['skeleton1'])
        new_skeleton2.append(skeletons[i]['skeleton2'])
        hooman2_frames.append(i)
        continue
    if len(skeletons[i])==1 and len(skeletons[i-1])==1:
        neigh.fit([[new_skeleton1[-1][0][1],  new_skeleton1[-1][1][1]]])
        dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        if dist[0][0] < 10:
            new_skeleton1.append(skeletons[i]['skeleton1'])


    if len(skeletons[i])==2 and len(skeletons[i-1])==1:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]]])
        dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        dist_2, nn_2 =  neigh.kneighbors([[skeletons[i]['skeleton2'][0][1],  skeletons[i]['skeleton2'][1][1]]], 
                                    return_distance = True)

        if dist_1 < dist_2 and dist_1 <10:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            new_skeleton2.append(skeletons[i]['skeleton2'])
            hooman2_frames.append(i)
        elif dist_2 < dist_1 and dist_2 <10:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            new_skeleton2.append(skeletons[i]['skeleton1'])
            hooman2_frames.append(i)



    if len(skeletons[i])==1 and len(skeletons[i-1])==2:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]], 
                [new_skeleton2[-1][0][1], new_skeleton2[-1][1][1]]])
        dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    return_distance = True)
        if nn[0][0] ==0 and dist[0][0] < 10:
            new_skeleton1.append(skeletons[i]['skeleton1'])
        elif nn[0][0] ==1 and dist[0][0] < 10:
            new_skeleton2.append(skeletons[i]['skeleton1'])


    if len(skeletons[i])==2 and len(skeletons[i-1])==2:
        neigh.fit([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]], 
                [new_skeleton2[-1][0][1], new_skeleton2[-1][1][1]]])
    
        
        dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]], 
                                    n_neighbors=1, return_distance = True)
      #  if i==26:
       #     print([[new_skeleton1[-1][0][1], new_skeleton1[-1][1][1]]], 
        #        [[new_skeleton2[-1][0][1], new_skeleton2[-1][1][1]]])
         #   print([[skeletons[i]['skeleton1'][0][1],  skeletons[i]['skeleton1'][1][1]]])
          #  print(dist_1, nn_1)
        
        dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][1],  skeletons[i]['skeleton2'][1][1]]], 
                                    return_distance = True)
     #   
      #      print(dist_1, nn_1, dist_2, nn_2)
       #     print(new_skeleton1[-1], new_skeleton2[-1])
            


        if nn_1[0][0] == 0 and dist_1[0][0] < 15:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            if dist_2[0][0] <15: ## making sure the other person hasn't suddenly switched to something/someone else
                new_skeleton2.append(skeletons[i]['skeleton2'])
                hooman2_frames.append(i)
        elif nn_1[0][0] == 1 and dist_1[0][0] < 15:
            new_skeleton2.append(skeletons[i]['skeleton1'])
            if dist_2[0][0] <15:
                new_skeleton1.append(skeletons[i]['skeleton2'])
                hooman2_frames.append(i)
        else:
            print("2 and 2", i)

    counter +=1

   # print(counter)
    hooman1.append(frames[i][round(np.nanmin(new_skeleton1[i][1])):round(np.nanmax(new_skeleton1[i][1])),
                        round(np.nanmin(new_skeleton1[i][0])):round(np.nanmax(new_skeleton1[i][0]))])

   # hooman2.append(frames[i][round(np.nanmin(new_skeleton2[i][1])):round(np.nanmax(new_skeleton2[i][1])),
    #                    round(np.nanmin(new_skeleton2[i][0])):round(np.nanmax(new_skeleton2[i][0]))])
            

        

In [ ]:
for hooman in hooman1:
    cv2.imshow('human1', hooman)
    cv2.waitKey(0) 

In [ ]:
hooman2=[]
for ind, skel in zip(hooman2_frames, new_skeleton2):
    hooman2.append(frames[ind][round(np.nanmin(skel[1])):round(np.nanmax(skel[1])),
                        round(np.nanmin(skel[0])):round(np.nanmax(skel[0]))])

In [ ]:
for hooman in hooman2:
    cv2.imshow('human2', hooman)
    cv2.waitKey(0) 

In [ ]:
len(frames)

In [ ]:
skeletons[25], skeletons[26]